In [ ]:
# Import modul
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import StaleElementReferenceException, ElementClickInterceptedException
import pandas as pd
import time

# Setup Firefox
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Firefox(options=options)

# List Kategori
labels = [
       'Accountant', 'Advocate', 'Agriculture', 'Apparel', 'Architecture',
       'Arts', 'Automobile', 'Aviation', 'Banking', 'BPO',
       'Building and Construction', 'Business Analyst', 'Civil Engineer',
       'Consultant', 'Data Science', 'Database', 'Designing',
       'Digital Media', 'DotNet Developer', 'Education',
       'Electrical Engineering', 'Finance',
       'Food and Beverages', 'Health and Fitness', 'Human Resources',
       'Information Technology', 'Java Developer', 'Management',
       'Network Security Engineer',
       'Operations Manager', 'PMO', 'Public Relations',
       'Python Developer', 'React Developer', 'Sales', 'SAP Developer',
       'SQL Developer', 'Testing', 'Web Designing'
]

all_jobs = []  # untuk menyimpan hasil dari semua kategori

# Loop untuk setiap label
for label in labels:
    print(f"\nScraping kategori: {label}")

    job_list = []
    seen_ids = set()

    # Buka halaman utama untuk kategori ini
    url = f"https://www.linkedin.com/jobs/search/?geoId=102478259&keywords={label.replace(' ', '%20')}"
    driver.get(url)
    time.sleep(8)
    # Scroll dan load sampai habis
    last_height = driver.execute_script("return document.body.scrollHeight")
    stagnant_scrolls = 0

    while True:
        # Scroll ke bawah untuk memicu lazy load
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

        # Klik tombol See more jobs jika ada
        try:
            see_more_bottons = driver.find_elements(By.XPATH, "//button[contains(., 'See more jobs')]")
            for btn in see_more_bottons:
                try:
                    driver.execute_script("arguments[0].click();", btn)
                    print("Klik See more jobs")
                    time.sleep(2)
                except ElementClickInterceptedException:
                    pass
        except Exception:
            pass

        # Cek apakah halaman sudah tidak bertambah tinggi
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            stagnant_scrolls += 1
        else:
            stagnant_scrolls = 0
        last_height = new_height

        # Jika sudah mentok (tidak berubah 2 kali berturut-turut)
        if stagnant_scrolls >= 2:
            print("Sudah mencapai akhir halaman.")
            break

    # Mengambil semua job card
    job_cards = driver.find_elements(By.CSS_SELECTOR, ".base-card, .job-card-container")
    print(f"Total job card ditemukan: {len(job_cards)}")

    for job in job_cards:
        try:
            job_id = job.get_attribute("data-entity-urn") or job.get_attribute("data-job-id")
            if not job_id or job_id in seen_ids:
                continue
            seen_ids.add(job_id)

            title = job.find_element(By.CSS_SELECTOR, "h3").text.strip()
            company = job.find_element(By.CSS_SELECTOR, "h4").text.strip()
            location = job.find_element(By.CLASS_NAME, "job-search-card__location").text.strip()
            date = job.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

            job_list.append([job_id, title, company, location, date, label])
        except Exception:
            continue

    print(f"Total job tersimpan untuk '{label}': {len(job_list)}")
    all_jobs.extend(job_list)

df = pd.DataFrame(all_jobs, columns=["job_id", "Title", "Company", "Location", "Date", "Category"])
df.drop_duplicates(subset=["job_id"], keep="first", inplace=True)

# Simpan file
OUTPUT_FILE_PATH = 'C:/Users/Acer/Trend Job/trend_job_list.csv' 
df.to_csv(OUTPUT_FILE_PATH, index=False, encoding="utf-8-sig")
print("\nSelesai scraping semua label!")
print(f"File tersimpan di: {OUTPUT_FILE_PATH}")
print(f"Total data unik tersimpan: {len(df)}")
